# Aspect based sentiment analyses part_01

## Постановка задачи

В начальной ноуте были расмотренны разные постановки задачи для отзывов об отелях. Но все постановки могли бы пренести bussiness value. И в тоже время, если посмотреть на данные, то видно, что решать задачу классификации, когда данные однородны, сомнительно. Так же это подтверждается тем, что ембеддинге не дают нам ощутимого семантического различия.
Отсюда сделан вывод, что  отзывы об отелях можно рассматривать в контексте постановки задачи aspect based sentiment analyses.

## Черновой дизайн решения

- Так как данные у нас не размечены, то сначал разметим часть данных. Размечать данные будем через платный api gpt-4, на первом этапе 100 сэмплов, с исползованием фреймворка LangChaing.
- Дальше, мы обучим open-source модель.
- Автоматическую категоризацию аспектов будем делать через модели класса sentence-transformers.
- Замерять качество будем случайным сэмплированием через gpt-4.  

## Выводы

- разметка через чат GPT-4 обошлась 6 долларов на 200 отзывов
- если бы рзамечали 6_000 отзывов, 180 долларов
- так как текст системного промпта значительный относительно текст отзыва, то можно в дальнейшем оптимизировать. 

## Следующий шаг

- оценить качество разметки

## Imports

In [1]:
import os
from dotenv import load_dotenv
from pathlib import Path

## Intialization

In [2]:
def find_project_root() -> Path:

    start_path = Path.cwd()
    for parent in start_path.parents:
        if (parent / ".git").exists() or (parent / "pyproject.toml").exists() or (parent / "setup.py").exists():
            return parent
    return start_path  # Fallback: if no marker is found, return the original path


# Get the project root automatically
PROJECT_ROOT = find_project_root()
print(Path.cwd())

d:\Projects\finam_hotel_reviews\notebooks


In [ ]:
ENV = os.getenv("ENV", "development")

# Загружаем переменные среды для режима разработки
if ENV == "development":
    env_path = Path(PROJECT_ROOT) / "configs" / ".env.development"
    try:
        load_dotenv(env_path, override=False)
        print("Environment variables loaded successfully")
    except Exception as e:
        print(f"Error loading environment variables: {e}")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

Environment variables loaded successfully


## Load data

In [ ]:
import pandas as pd

df = pd.read_excel(PROJECT_ROOT / "data" / "raw" / "Test_data_marked.xlsx")
df.head(2)

,review_id,review_text
0,1,Элегантный отель в самом центре Дубай. Красивы...
1,2,Если стиль вашей поездки в Дубай-- это шоппинг...


In [56]:
n_train_samples = 200
df_train = df.sample(n_train_samples)
df_ootrain = df.drop(df_train.index)

len(df), len(df_train), len(df_ootrain)

(6876, 200, 6676)

In [58]:
reviews = df_train.review_text.to_list()
reviews[:2]

['Это был самый маленький по площади номер из всех отелей, где я останавливалась. Тем не менее, очень уютненький и комфортабельный мини отельчик. Мебель новая, все функционирует, удобно расположен. Учитывая дороговизну отелей Парижа, для эконом варианта вполне достойно.',
 'Понравилось всё!!! От персонала до оснащения номера!!! Модно,стильно,уютно. Огромный плазменный телевизор в номере, наличие банных принадлежностей (для мини-отеля это редкость), любезно предоставленная посуда,которая понадобилась в ходе проживания, ванная комната с просто блестящей сантехникой - всё это сложило невероятно приятное впечатление о гостинице!!! Девушка на ресепшене встретила с улыбкой, всё подробно объяснила, посоветовала. Желаю отелю дальнейшего процветания!Огромное спасибо за возможность прекрасно провести время!']

## Make langchain and create prompt

In [59]:
import os
import pandas as pd
from tqdm import tqdm
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnableSequence
import json

# Устанавливаем API-ключ OpenAI
os.environ["OPENAI_API_KEY"] = "your_openai_api_key"

# Подключаем ChatGPT-4
llm = ChatOpenAI(api_key=OPENAI_API_KEY, model="gpt-4", temperature=0, top_p=0)


# Шаблон промпта для разметки
prompt = PromptTemplate(
    input_variables=["review"],
    template="""
Ты - аналитие по анализу отзывов. Разметь данный отзыв об отеле, выделяя все возможные аспекты и их тональность.

Примеры:
Отзыв: "Отличное расположение, но номера маленькие. Завтрак был вкусным, но дорогим."
Ответ:
[
  {{ "aspect": "Расположение", "sentiment": "positive" }},
  {{ "aspect": "Номер", "sentiment": "negative" }},
  {{ "aspect": "Завтрак", "sentiment": "mixed" }}
]

Отзыв: "Очень грязный номер, но персонал был дружелюбным."
Ответ:
[
  {{ "aspect": "Чистота", "sentiment": "negative" }},
  {{ "aspect": "Персонал", "sentiment": "positive" }}
]

Теперь разметь следующий отзыв:

Отзыв: {review}

Формат ответа:
[
  {{ "aspect": "Расположение", "sentiment": "positive" }},
  {{ "aspect": "Номер", "sentiment": "negative" }}
]
""",
)

# Создаем цепочку
chain = prompt | llm


df = pd.DataFrame({"review_id": range(1, len(reviews) + 1), "text": reviews})


def label_review(review):
    response = chain.invoke({"review": review})
    response_text = response.content  # Получаем текст из AIMessage
    try:
        return json.loads(response_text)  # Преобразуем строку в JSON
    except json.JSONDecodeError:
        return []  # В случае ошибки возвращаем пустой список


# Размечаем данные
tqdm.pandas()
df["labels"] = df["text"].progress_apply(label_review)


def normalization(label_data):

    for entry in label_data:
        entry["aspect"] = entry["aspect"].lower()

    return label_data


# Применяем обратную замену аспектов
df["labels"] = df["labels"].apply(normalization)

# Выводим результат
df.head(2)

100%|██████████| 200/200 [22:06<00:00,  6.63s/it]


,review_id,text,labels
0,1,Это был самый маленький по площади номер из вс...,"[{'aspect': 'размер номера', 'sentiment': 'neg..."
1,2,Понравилось всё!!! От персонала до оснащения н...,"[{'aspect': 'персонал', 'sentiment': 'positive..."


In [ ]:
# Save labeled results
df.to_excel(PROJECT_ROOT / "data" / "interim" / "200_labeled_gpt_4.xlsx")
df.to_csv(PROJECT_ROOT / "data" / "interim" / "200_labeled_gpt_4.csv")